Notebook to develope large document corpus vector index

In [ ]:
from datasets import load_dataset
from pprint import pprint
# Load the KILT Wikipedia dataset
dataset = load_dataset('kilt_wikipedia')

# Display a sample of the dataset
print(dataset)

In [ ]:
# Keep only 'text' and 'id' columns
print(dataset.column_names)
dataset = dataset["full"]
filtered_dataset = dataset.remove_columns([col for col in dataset.column_names if col not in ['text', 'kilt_id']])

# Show the structure after filtering
print(filtered_dataset[0])
print(len(filtered_dataset))

In [ ]:
slice_for_dev=100
kilt_list = []
for ID, Text in zip(filtered_dataset[:slice_for_dev]['kilt_id'], filtered_dataset[:slice_for_dev]['text']):
    # print(f'ID: {ID}')
    paragraphs = [elem.strip() for elem in Text["paragraph"]]
    raw_text = ' '.join(paragraphs)
    raw_text = raw_text.replace('\n', ' ')
    raw_text = raw_text.replace('BULLET::::', 'Bullet:')
    raw_text = raw_text.replace('SECTION::::', 'Section:')
    # print(f'Text: {raw_text}')

    print('---')
    kilt_list.append({'text': raw_text, 'id': ID})
kilt_list

In [ ]:
# Make new marqo index and add the new data


import sys
import time
from pprint import pprint

sys.path.append("../dev/")
sys.path.append("../src/")
from dataset_helpers import DatasetHelpers
from vector_store import VectorStore

# Define API ENDPOINTS
LLM_URL = "http://10.103.251.104:8040/v1"
LLM_NAME = "llama3"
MARQO_URL = "http://10.103.251.104:8882"
MARQO_URL_GPU = "http://10.103.251.104:8880"

##
## Load Dataset
##

corpus_list = kilt_list  # List of documents to index
##
## Load the VectorStore
##
documentDB_GPU = VectorStore(MARQO_URL_GPU)  # Connect to marqo client via python API
print(documentDB_GPU.getIndexes())  # Print all indexes


##
## Delete old index
##

# documentDB_GPU.deleteIndex("KILT-test")  # Delete the index


##
## Create new index
##

index_settings = {
    "split_length": 0,  # Number of elmenents in a split
    "split_method": "sentence",  # Method of splitting
    "split_overlap": 0,  # Number of overlapping tokens in a split
    "distance_metric": "prenormalized-angular",  # Distance metric for ann
    "model": "flax-sentence-embeddings/all_datasets_v4_mpnet-base",  # Model for vector embedding
}

documentDB_GPU.createIndex("KILT-test", index_settings)  # Create a new index


##
## Index documents
##


maxDocs = 100000  # Number of documents to index
documentDB_GPU.connectIndex("KILT-test")  # Connect to the minibio
start = time.time()
documentDB_GPU.indexDocuments(corpus_list, maxDocs)  # Add documents to the index
end = time.time()
## Time for indexing 100 documents on marqo with cpu
print(f"Time for indexing {maxDocs} documents: {end - start} seconds")


print(documentDB_GPU.getIndexStats())  # Print index stats

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('flax-sentence-embeddings/all_datasets_v4_mpnet-base')
text = "Replace me by any text you'd like."
text_embbedding = model.encode(text)
print(text_embbedding.shape)

/home/simon/master_project/software/venv_koenigsi/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


(768,)
